In [18]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix,f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pickle
import re
from os import path
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

def process_tweet(tweet):
    return " ".join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])", " ",tweet.lower()).split())

def save_model(name, model):
    with open(name, 'wb') as fid:
        pickle.dump(model, fid) 
        
def load_model(name):
    with open(name, 'rb') as fid:
        return pickle.load(fid)


In [20]:
hatePath = './../Datasets/hatevsNoHate.txt'
df = pd.read_csv(hatePath)
hate_tweet = df[df.label == 1]
normal_tweet = df[df.label == 0]

text = " ".join(review for review in hate_tweet.tweet)

df_Stat=df[['label','tweet']].groupby('label').count().reset_index()
df_Stat.columns=['label','count']
df_Stat['percentage']=(df_Stat['count']/df_Stat['count'].sum())*100
df_Stat

df['processed_tweets'] = df['tweet'].apply(process_tweet)

cnt_non_fraud = df[df['label'] == 0]['processed_tweets'].count()
df_class_fraud = df[df['label'] == 1]
df_class_nonfraud = df[df['label'] == 0]
df_class_fraud_oversample = df_class_fraud.sample(cnt_non_fraud, replace=True)
df_oversampled = pd.concat([df_class_nonfraud, df_class_fraud_oversample], axis=0)

print('Random over-sampling:')
print(df_oversampled['label'].value_counts())

X = df_oversampled['processed_tweets']
y = df_oversampled['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = None)

count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)

x_train_counts = count_vect.fit_transform(X_train)
x_train_tfidf = transformer.fit_transform(x_train_counts)

print(x_train_counts.shape)
print(x_train_tfidf.shape)

x_test_counts = count_vect.transform(X_test)
x_test_tfidf = transformer.transform(x_test_counts)


Random over-sampling:
0    39500
1    39500
Name: label, dtype: int64
(63200, 42471)
(63200, 42471)


In [43]:
print(x_test_counts)

  (0, 5083)	1
  (0, 5403)	1
  (0, 8243)	1
  (0, 13538)	1
  (0, 14632)	1
  (0, 15956)	1
  (0, 20801)	1
  (0, 34079)	1
  (0, 36387)	1
  (0, 41365)	1
  (1, 17432)	1
  (1, 32125)	1
  (2, 9428)	1
  (2, 10741)	1
  (2, 16906)	1
  (2, 18474)	1
  (2, 28928)	1
  (2, 35726)	1
  (2, 39831)	1
  (3, 1683)	1
  (3, 2700)	1
  (3, 8039)	1
  (3, 10494)	1
  (3, 20005)	1
  (3, 20340)	1
  :	:
  (15797, 30482)	1
  (15797, 30670)	1
  (15797, 33094)	1
  (15797, 40381)	1
  (15798, 630)	1
  (15798, 2051)	1
  (15798, 2402)	1
  (15798, 5283)	1
  (15798, 8356)	1
  (15798, 14564)	1
  (15798, 23274)	1
  (15798, 34929)	1
  (15798, 41082)	1
  (15799, 3201)	1
  (15799, 6631)	1
  (15799, 8501)	1
  (15799, 13497)	1
  (15799, 18116)	1
  (15799, 18129)	1
  (15799, 18927)	1
  (15799, 22581)	1
  (15799, 23549)	1
  (15799, 29824)	1
  (15799, 39104)	1
  (15799, 42012)	1


In [11]:
from sklearn import svm
lin_clf = svm.LinearSVC()
lin_clf.fit(x_train_tfidf,y_train)



LinearSVC()

In [12]:
predict_svm = lin_clf.predict(x_test_tfidf)
predict_svm


array([1, 0, 1, ..., 1, 0, 0], dtype=int64)

In [48]:
a = np.array(["I hate people"])

count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)

b = count_vect.fit_transform(a)
print(b)
b = transformer.fit_transform(b)
print(b)

model = load_model('svm.pkl')
x = model.predict(b.reshape(1, -1))

  (0, 0)	1
  (0, 1)	1
  (0, 1)	0.7071067811865475
  (0, 0)	0.7071067811865475


ValueError: X has 2 features per sample; expecting 42380